# Prepare grids for gravity inversion

In [1]:
import pygmt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
os.getcwd()

'/Users/home/tankerma/RIS_gravity_inversion/inversion_layers'

In [2]:
figheight=90 # in mm
"""
# for Ross Embayment
ris_xl= -760_000
ris_yl= -2_200_000 #N extent
ris_xh= 600_000
ris_yh= -350_000 #S extent
"""
# for RIS only
ris_xl=-960_000
ris_xh=800_000
ris_yl=-1_800_000
ris_yh=-40_000

figwidth=figheight*(ris_xh-ris_xl)/(ris_yh-ris_yl)
risratio = (ris_yh - ris_yl) / (figheight/1000)
risreg = str(ris_xl) + '/' + str(ris_xh) + '/' + str(ris_yl) + '/' + str(ris_yh) #W/E/S/N
risproj = "x1:" + str(risratio)
antreg = "-3330000/1900000/-3330000/1850000"
risproj_ll = "s0/-90/-71/1:" + str(risratio)

def make_fig():
    global fig, pos_str
    fig = pygmt.Figure()
    with pygmt.config(MAP_FRAME_TYPE = 'inside'):
        fig.coast(region = risreg, projection = risproj_ll, land = 'white', water = 'white', frame = ["nwse", "xf100000", "yf100000", "g0"],verbose='e')
    # set colorbar params
    barwidth = figwidth/10 * 0.4 # color bar width is 40% of the map width, in cm
    pos_str = 'jTL+w' + str(barwidth) + 'c+jTL+o1c/0.5c+h+ml+e'
    # with pygmt.config(FONT_ANNOT_PRIMARY = '4p,white', FONT_LABEL = '6p,white', 
    #                   MAP_ANNOT_MIN_SPACING = '0p', MAP_ANNOT_OFFSET_PRIMARY = '0.5p', MAP_TICK_PEN_PRIMARY = '0.4p,white', 
    #                   MAP_TICK_LENGTH_PRIMARY = '2p', MAP_FRAME_PEN = '0.3p,white', MAP_LABEL_OFFSET = '4p')


In [ ]:
!gmt grdsample -R-760000/600000/-1600000/-240000 -I20e3 ROSETTA_gravity.nc -GROSETTA_gravity_20k.nc
!gmt grdsample -R-760000/600000/-1600000/-240000 -I20e3 BedMachine_surface_5k_filled.nc -GBedMachine_surface_filled_20k.nc
!gmt grdsample -R-760000/600000/-1600000/-240000 -I20e3 BedMachine_icebase_5k_filled.nc -GBedMachine_icebase_filled_20k.nc
!gmt grdsample -R-760000/600000/-1600000/-240000 -I20e3 BedMachine_bed_5k.nc -GBedMachine_bed_20k.nc 
!gmt grdsample -R-760000/600000/-1600000/-240000 -I20e3 ROSETTA_basement_BedMachine_bed.nc -GROSETTA_basement_BedMachine_bed_20k.nc 

!gmt grd2xyz -R-760000/600000/-1600000/-240000 ROSETTA_gravity_20k.nc > ROSETTA_gravity_20k.csv
!gmt grd2xyz -R-760000/600000/-1600000/-240000 BedMachine_surface_filled_20k.nc > BedMachine_surface_filled_20k.csv
!gmt grd2xyz -R-760000/600000/-1600000/-240000 BedMachine_icebase_filled_20k.nc > BedMachine_icebase_filled_20k.csv
!gmt grd2xyz -R-760000/600000/-1600000/-240000 BedMachine_bed_20k.nc > BedMachine_bed_20k.csv
!gmt grd2xyz -R-760000/600000/-1600000/-240000 ROSETTA_basement_BedMachine_bed_20k.nc > ROSETTA_basement_BedMachine_bed_20k.csv

ANTASed Model

In [ ]:

file ='D:/Datasets/ANTASed/0.1_lim_b.dat'
df = pd.read_csv(file, header=None, 
                delim_whitespace=True, 
                names=['x_100km', 'y_100km', 'thick_km'])

df['x'], df['y'], df['thick'] = df.x_100km*100000, df.y_100km*100000, df.thick_km*1000

pygmt.xyz2grd(data=df[['x','y','thick']], region=antreg, spacing = 10e3, outgrid='D:/Datasets/ANTASed/ANTASed_thick.nc')

In [ ]:
!gmt grdsample D:/Datasets/ANTASed/ANTASed_thick.nc -R-1060000/900000/-1900000/60000 -r -I10000+e -GD:/Datasets/ANTASed/ANTASed_thick_10k.nc
!gmt grdsample bedmachine/BedMachine_bed_5k.nc -R-1060000/900000/-1900000/60000 -r -I10000+e -Gbedmachine/BedMachine_bed_10k.nc
!gmt grdmath -Df bedmachine/BedMachine_bed_10k.nc D:/Datasets/ANTASed/ANTASed_thick_10k.nc SUB = D:/Datasets/ANTASed/ANTASed_basement.nc
!gmt grdinfo D:/Datasets/ANTASed/ANTASed_basement.nc

In [ ]:
make_fig()
fig.grdimage(grid='D:/Datasets/ANTASed/ANTASed_basement.nc', cmap='viridis', projection=risproj, region=risreg)
fig.colorbar(cmap = True, position = pos_str,frame = 'xa3000+l"ANTASed basement elevation [km]"') #, box = '+gblack+p0.5p,black+c3p')
fig.show()

In [ ]:
!gmt grdsample Ross_Embayment_basement_filt.nc -I10000+e -R-1060000/900000/-1900000/60000 -GRoss_Embayment_basement_filt_resample.nc
!gmt grdblend Ross_Embayment_basement_filt_resample.nc D:/Datasets/ANTASed/ANTASed_basement.nc -Cf -GANTASed_mag_basement_merged.nc
!gmt grdinfo ANTASed_mag_basement_merged.nc

In [ ]:
make_fig()
fig.grdimage(grid='ANTASed_mag_basement_merged.nc', cmap='viridis', projection=risproj, region=risreg)
fig.colorbar(cmap = True, position = pos_str,frame = 'xa3000+l"ANTASed basement elevation [km]"') #, box = '+gblack+p0.5p,black+c3p')
fig.show()

In [ ]:
!gmt grd2xyz Ross_Embayment_basement_filt.nc -s > Ross_Embayment_basement_filt.xyz
pygmt.surface(data = 'Ross_Embayment_basement_filt.xyz', 
              spacing='10000+e', 
              region='-1060000/900000/-1900000/60000',
              M='10c',
              outgrid='RE_basement_buffer.nc', verbose='e') 
!gmt grdsample Ross_Embayment_basement_filt.nc -I10000+e -R-1060000/900000/-1900000/60000 -rp -GRoss_Embayment_basement_filt_resample.nc
!gmt grdsample RE_basement_buffer.nc -I10000+e -R-1060000/900000/-1900000/60000 -rp -GRE_basement_buffer.nc
!gmt grdmath -Df RE_basement_buffer.nc ISNAN = RE_basement_buffer_mask.nc

!gmt grdmath -Df RE_basement_buffer_mask.nc D:/Datasets/ANTASed/ANTASed_basement.nc MUL = D:/Datasets/ANTASed/ANTASed_basement_buffer.nc
!gmt grdmath D:/Datasets/ANTASed/ANTASed_basement_buffer.nc 0 NAN = D:/Datasets/ANTASed/ANTASed_basement_buffer_mask.nc
!gmt grdblend Ross_Embayment_basement_filt_resample.nc D:/Datasets/ANTASed/ANTASed_basement_buffer_mask.nc -Cf -GANTASed_mag_basement.nc
# interpolated gap in QGIS with GRASS r.fillnulls -> ANTASed_mag_basement_filled.nc
!gmt grdinfo ANTASed_mag_basement.nc

In [ ]:
make_fig()
fig.grdimage(grid='ANTASed_mag_basement_filled.nc', cmap='viridis', projection=risproj, region=risreg)
fig.colorbar(cmap = True, position = pos_str,frame = 'xa3000+l"ANTASed basement elevation [km]"') #, box = '+gblack+p0.5p,black+c3p')
fig.show()

## SURFACE

#### Bedmap2 surface, NaN's set to 0

In [19]:
fname='/Volumes/arc_04/tankerma/Datasets/bedmap2/bedmap2_tiff/bedmap2_surface.tif'
fname2='/Volumes/arc_04/tankerma/Datasets/bedmap2/bedmap2_tiff/gl04c_geiod_to_WGS84.tif'
# not sure if should conver to WGS than fill nans with 0, or vice-versa
!gmt grdmath {fname} {fname2} ADD = bedmap2_surface_WGS84.nc
!gmt grdmath bedmap2_surface_WGS84.nc 0 DENAN = bedmap2_surface_WGS84_filled.nc

In [ ]:
!gmt grdmath -Df bedmap2_surface.tif gl04c_geoid_to_WGS84.tif ADD = bedmap2_surface_WGS_84.tif

In [ ]:
!gmt grdsample bedmap2_surface_WGS_84.tif -Gbedmap2_surface_WGS_84_RIS_5k.nc -R-760000/600000/-1600000/-240000 -I5000+e

In [ ]:
!gmt grdmath -Df bedmap2_surface_WGS_84_RIS_5k.nc 0 DENAN = layers/bedmap2_surface_WGS_84_RIS_5k_filled.nc

In [ ]:
!gmt grd2xyz bedmap2_surface_WGS_84_RIS_5k.nc  > bedmap2_surface_WGS_84_RIS_5k.csv

In [ ]:
make_fig()
fig.grdimage(grid ='layers/bedmap2_surface_WGS_84_RIS_5k_filled.nc', cmap ='globe', shading='grdgradient+a45/45+nt1+m0', projection=risproj, region=risreg, nan_transparent=True)
fig.colorbar(cmap ='globe', position = pos_str,frame = 'xa2000+l"Bed Elevation [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()

## ICEBASE

#### Bedmap2 surface-thickness, NaN's set to 0

!gmt grdmath -Df bedmap2_surface.tif bedmap2_thickness.tif SUB = bedmap2_icebase.nc

!gmt grdsample bedmap2_icebase.nc -Gbedmap2_icebase_RIS.nc -R-760000/600000/-1600000/-240000

!gmt grdmath -Df bedmap2_icebase.nc gl04c_geiod_to_WGS84.tif ADD = bedmap2_icebase_WGS_84.tif

!gmt grdsample bedmap2_icebase_WGS_84.tif -Gbedmap2_icebase_WGS_84_RIS_5k.nc -R-760000/600000/-1600000/-240000 -I5000+e

!gmt grdmath -Df bedmap2_icebase_WGS_84_RIS_5k.nc 0 DENAN = layers/bedmap2_icebase_WGS_84_RIS_5k_filled.nc

!gmt grd2xyz bedmap2_icebase_WGS_84_RIS_5k_filled.nc > bedmap2_icebase_WGS_84_RIS_5k_filled.csv

In [ ]:
!gmt grdsample bedmap2_thickness.tif -Gbedmap2_thickness_RIS_5k.nc -R-760000/600000/-1600000/-240000 -I5000+e

In [ ]:
!gmt grdmath -Df layers/bedmap2_surface_WGS_84_RIS_5k_filled.nc bedmap2_thickness_RIS_5k.nc SUB = bedmap2_icebase_WGS_84_RIS_5k.nc

In [ ]:
!gmt grdmath -Df bedmap2_icebase_WGS_84_RIS_5k.nc 0 DENAN = layers/bedmap2_icebase_WGS_84_RIS_5k_filled.nc

In [ ]:
make_fig()
fig.grdimage(grid ='layers/bedmap2_icebase_WGS_84_RIS_5k_filled.nc', cmap ='globe', region=risreg, projection=risproj, shading='grdgradient+a45/45+nt1+m0', nan_transparent=True)
fig.colorbar(cmap ='globe', position = pos_str,frame = 'xa2000+l"Ice Base [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()

## BED

#### Bedmap2 bed

In [ ]:
!gmt grdmath -Df bedmap2_bed.tif gl04c_geiod_to_WGS84.tif ADD = bedmap2_bed_WGS_84.tif

In [ ]:
!gmt grdsample bedmap2_bed_WGS_84.tif -Glayers/bedmap2_bed_WGS_84_RIS_5k.nc -R-760000/600000/-1600000/-240000 -I5000+e

In [ ]:
!gmt grd2xyz layers/bedmap2_bed_WGS_84_RIS_5k.tif > bedmap2_bed_WGS_84_RIS_5k.csv

In [ ]:
make_fig()
fig.grdimage(grid ='layers/bedmap2_bed_WGS_84_RIS_5k.nc', cmap ='globe', projection=risproj, shading='grdgradient+a45/45+nt1+m0', region=risreg, nan_transparent=True)
fig.colorbar(cmap ='globe', position = pos_str,frame = 'xa2000+l"Bed Elevation [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()

#### Bedmap2 bed minus bedmachine bed

In [ ]:
!gmt grdmath -Df layers/bedmap2_bed_WGS_84_RIS_5k.nc layers/BedMachine_bed_5k.nc SUB = layers/tmp_bedmap2_bedmachine_bed_diff.nc 

In [ ]:
make_fig()
pygmt.makecpt(cmap='polar', series='-200/200', background='o')
fig.grdimage(grid ='layers/tmp_bedmap2_bedmachine_bed_diff.nc', cmap =True, projection=risproj, shading='grdgradient+a45/45+nt1+m0', region=risreg, nan_transparent=True)
fig.colorbar(cmap =True, position = pos_str,frame = 'xa50+l"Bed Elevation Difference [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()

## BASEMENT

#### ROSETTA_lindeque basement, merged to bedmap2 bed

## SURFACE

#### Bedmachine surface, NaN's set to 0

In [ ]:
!gmt grdsample D:/Bedmachine2/BedMachineAntarctica_2020-07-15_v02.nc?surface -R-1060000/900000/-1900000/60000 -r -Gbedmachine/BedMachine_surface.nc -I5000+e

In [ ]:
!gmt grdmath -Df bedmachine/BedMachine_surface.nc 0 DENAN = bedmachine/BedMachine_surface_5k_filled.nc

In [ ]:
make_fig()
fig.grdimage(grid ='bedmachine/BedMachine_surface_5k_filled.nc', cmap ='globe', shading='grdgradient+a45/45+nt1+m0', projection=risproj, region=risreg, nan_transparent=True)
fig.colorbar(cmap ='globe', position = pos_str,frame = 'xa2000+l"Bed Elevation [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()

## ICEBASE

#### BedMachine surface-thickness, NaN's set to 0

In [ ]:
!gmt grdsample D:/Bedmachine2/BedMachineAntarctica_2020-07-15_v02.nc?thickness -R-1060000/900000/-1900000/60000 -r -Gbedmachine/BedMachine_thickness.nc -I5000+e

In [ ]:
!gmt grdmath bedmachine/BedMachine_surface.nc bedmachine/BedMachine_thickness.nc SUB = bedmachine/BedMachine_icebase_5k.nc

In [ ]:
!gmt grdmath -Df bedmachine/BedMachine_icebase_5k.nc 0 DENAN = bedmachine/BedMachine_icebase_5k_filled.nc

In [ ]:
make_fig()
fig.grdimage(grid ='bedmachine/BedMachine_icebase_5k_filled.nc', cmap ='globe', region=risreg, projection=risproj, shading='grdgradient+a45/45+nt1+m0', nan_transparent=True)
fig.colorbar(cmap ='globe', position = pos_str,frame = 'xa1000+l"Bed Elevation [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()

## BED

#### BedMachine bed

In [ ]:
!gmt grdsample D:/Bedmachine2/BedMachineAntarctica_2020-07-15_v02.nc?bed -Gbedmachine/BedMachine_bed_5k.nc -R-1060000/900000/-1900000/60000 -r -I5000+e

In [ ]:
make_fig()
fig.grdimage(grid ='bedmachine/BedMachine_bed_5k.nc', cmap ='globe', projection=risproj, shading='grdgradient+a45/45+nt1+m0', region=risreg, nan_transparent=True)
fig.colorbar(cmap ='globe', position = pos_str,frame = 'xa1000+l"Bed Elevation [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()

## BASEMENT

#### ROSETTA_lindeque basement, merged to BedMachine bed

In [ ]:
!gmt grd2xyz Ross_Embayment_basement_filt.nc -s > Ross_Embayment_basement_filt.xyz
pygmt.surface(data = 'Ross_Embayment_basement_filt.xyz', 
              spacing='5000+e', 
              region='-1060000/900000/-1900000/60000',
              M='20c',
              outgrid='RE_basement_buffer.nc', verbose='e') 
!gmt grdsample Ross_Embayment_basement_filt.nc -I5000+e -R-1060000/900000/-1900000/60000 -rp -GRoss_Embayment_basement_filt_resample.nc
!gmt grdsample RE_basement_buffer.nc -I5000+e -R-1060000/900000/-1900000/60000 -rp -GRE_basement_buffer.nc
!gmt grdmath -Df RE_basement_buffer.nc ISNAN = RE_basement_buffer_mask.nc
!gmt grdmath -Df RE_basement_buffer_mask.nc bedmachine/BedMachine_bed_5k.nc MUL = bedmachine/BedMachine_bed_5k_buffer.nc
!gmt grdmath bedmachine/BedMachine_bed_5k_buffer.nc 0 NAN = bedmachine/BedMachine_bed_5k_buffer_mask.nc
!gmt grdblend Ross_Embayment_basement_filt_resample.nc bedmachine/BedMachine_bed_5k_buffer_mask.nc -Cf -GRE_basement_BedMachinebed.nc
# interpolated gap in QGIS with GRASS r.fillnulls -> RE_basement_bed_filled.tif
!gmt grdinfo Ross_Embayment_basement_filt_resample.nc

In [ ]:
!gmt grdinfo bedmachine/BedMachine_bed_5k_buffer_mask.nc

In [ ]:
make_fig()

#pygmt.grdfilter(grid='rosetta_lindeque_basement.tif', filter='g100000', distance='0', outgrid='rosetta_lindeque_basement_filt.tif')

fig.grdimage(grid='bedmachine/ROSETTA_basement_BedMachine_bed_merged.nc', cmap='globe', projection=risproj, region=risreg, shading='grdgradient+a45/45+nt1+m0', nan_transparent=True, dpi='300')
fig.colorbar(cmap ='globe', position = pos_str,frame = 'xa1000+l"Depth to Basement [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()

## MOHO

### An

In [9]:
fname='/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd'
# print(pygmt.grdinfo(fname))
grid = pygmt.load_dataarray(fname)
grid = pygmt.grdproject(grid, projection='EPSG:3031')
grid = grid*1000
crust = pygmt.grdsample(grid, region='-1060000/900000/-1900000/60000', spacing=10e3, registration='p')

/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd: Title: region_mask.grd
/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd: Command: grdmath region_mask.grd plot.grd MUL = plot_new.grd
/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd: Remark: 
/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd: Gridline node registration used [Geographic grid]
/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd: Grid file format: nf = GMT netCDF format (32-bit float), CF-1.7
/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd: x_min: -180 x_max: 180 x_inc: 0.0666666666667 (4 min) name: longitude n_columns: 5401
/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd: y_min: -90 y_max: -24 y_inc: 0.0666666666667 (4 min) name: latitude n_rows: 991
/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd: v_min: 5.6293926239 v_max: 65.1211013794 name: z
/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-CRUST.grd: scale_facto

grdproject [WARNING]: x_inc does not divide 180; geographic boundary condition changed to natural.
/tmp/ipykernel_2287747/1648343706.py:6: SyntaxWarning: Short-form parameter (r) is not recommended. Use long-form parameter 'registration' instead.
  crust = pygmt.grdsample(grid, region='-1060000/900000/-1900000/60000', spacing=10e3, r='p')
grdsample [WARNING]: Output sampling interval in x exceeds input interval and may lead to aliasing.


In [20]:
bedmap2_surface = pygmt.grdsample('bedmap2_surface_WGS84_filled.nc', region='-1060000/900000/-1900000/60000', spacing=10e3)
moho = bedmap2_surface - crust
print(pygmt.grdinfo(moho))
fname='/Volumes/arc_04/tankerma/Datasets/An_et_al_2015/AN1-MOHO.nc'
moho.to_netcdf(fname)
# print(pygmt.grdinfo(fname))
# grid.plot()

grdsample [WARNING]: Output sampling interval in x exceeds input interval and may lead to aliasing.
grdsample [WARNING]: Output sampling interval in y exceeds input interval and may lead to aliasing.


: Title: 
: Command: 
: Remark: 
: Gridline node registration used [Cartesian grid]
: Grid file format: nf = GMT netCDF format (32-bit float), CF-1.7
: x_min: -1055000 x_max: 895000 x_inc: 10000 name: x n_columns: 196
: y_min: -1895000 y_max: 55000 y_inc: 10000 name: y n_rows: 196
: v_min: -53232.6054688 v_max: -7489.875 name: z
: scale_factor: 1 add_offset: 0
: format: classic



In [21]:
moho = pygmt.load_dataarray(fname)
print(pygmt.grdinfo(moho))

: Title: 
: Command: 
: Remark: 
: Gridline node registration used [Cartesian grid]
: Grid file format: nf = GMT netCDF format (32-bit float), CF-1.7
: x_min: -1055000 x_max: 895000 x_inc: 10000 name: x n_columns: 196
: y_min: -1895000 y_max: 55000 y_inc: 10000 name: y n_rows: 196
: v_min: -53232.6054688 v_max: -7489.875 name: z
: scale_factor: 1 add_offset: 0
: format: classic



### Shen

In [ ]:
Moho_Shen = pd.read_csv('Shen_moho.final.dat', header=None, sep='\ ', engine='python')
Moho_Shen.columns =['Lon','Lat','z']
Moho_Shen_uncert = pd.read_csv('Shen_un.moho.final.dat', header=None, sep='\ ', engine='python')
Moho_Shen_uncert.columns =['Lon','Lat','uncert']
Moho_Shen['uncert']=Moho_Shen_uncert.uncert
Moho_Shen.z=Moho_Shen.z.apply(lambda x:x*-1000)
Moho_Shen.uncert=Moho_Shen.uncert.apply(lambda x:x*1000)

moho = pygmt.surface(x=Moho_Shen.Lon, y=Moho_Shen.Lat, z=Moho_Shen.z, region='-180/180/-90/-60', spacing='10m', M='1c')
moho.plot()

In [ ]:
# moho = pygmt.grdproject(moho, projection='EPSG:3031')
moho = pygmt.grdsample(moho, region='-1060000/900000/-1900000/60000', spacing=20e3)

moho.plot()

### Lamb

In [ ]:
!gmt grdsample Crust_Lamb.nc -GCrust_Lamb_RIS.nc -R-760000/600000/-1600000/-240000 -T

In [ ]:
!gmt grdsample Crust_Lamb.nc -GCrust_Lamb_RE.nc -T

In [ ]:
!gmt grdmath -Df -R-760000/600000/-1600000/-240000 bedmap2_bed_WGS_84_RIS_5k.tif Crust_Lamb_RIS.nc  SUB = Lamb_Moho_wgs84_RIS.tif 

In [ ]:
!gmt grdmath -Df bedmap2_bed_WGS_84_5k.tif Crust_Lamb_RE.nc  SUB = Lamb_Moho_wgs84.tif 

In [ ]:
!gmt grd2xyz Lamb_Moho_wgs84_RIS.tif  >Lamb_Moho_wgs84_RIS.csv

In [ ]:
make_fig()

fig.grdimage(grid='Lamb_Moho_wgs84.tif', cmap="Moho.cpt", nan_transparent=True, dpi='500')
fig.colorbar(cmap="Moho.cpt", position = pos_str, frame = 'xa10000+l"Moho Depth [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()



## GRAVITY

In [ ]:
ris_xl= -760_000
ris_yl= -1600_000 #N extent
ris_xh= 600_000
ris_yh= -350_000 #S extent
risreg = str(ris_xl) + '/' + str(ris_xh) + '/' + str(ris_yl) + '/' + str(ris_yh) #W/E/S/N
aerograv_data = "D:/ROSETTA_Data/from_LDEO_2021/rs_2019_grav.csv"
gravity = pd.read_csv(aerograv_data)
gravity = pygmt.blockmedian(table=gravity[["x", "y", "FAG_levelled"]], 
                                         spacing="5000+e", 
                                         region=risreg)
pygmt.surface(x=gravity.x, y=gravity.y, z=gravity.FAG_levelled,
              spacing='5000+e',
              outfile="layers/ROSETTA_gravity.nc",
              region=risreg,) 
!gmt grdmask ..\shapefiles\RIS_outline.shp -GRIS_mask.nc -I5000+e -R-760000/600000/-1600000/-350000 -NNaN/1/1
!gmt grdmath -Df layers/ROSETTA_gravity.nc RIS_mask.nc MUL = layers/ROSETTA_gravity.nc
!gmt grdsample layers/ROSETTA_gravity.nc -Glayers/ROSETTA_gravity.nc -T

In [ ]:
!gmt grd2xyz layers/ROSETTA_gravity.nc > layers/ROSETTA_gravity.csv

In [ ]:
print(pygmt.grdinfo("GRAV_50k.nc"))

In [ ]:
!gmt xyz2grd antgg_FAG_10k_RIS_hole_filled.xyz -GGRAV_10k.nc -I10000+e -R-760000/600000/-1600000/-240000

In [ ]:
!gmt grdsample GRAV_10k.nc -GGRAV_50k.nc -R-760000/600000/-1600000/-240000 -I50000+e -T

In [ ]:
!gmt grd2xyz GRAV_50k.nc > GRAV_50k_fromgrid.csv

In [ ]:
make_fig()

fig.grdimage(grid='GRAV_50k.nc', cmap="wysiwyg", nan_transparent=True, dpi='500')
fig.colorbar(cmap=True, position = pos_str, frame = 'xa10000+l"Moho Depth [m]"', box = '+gblack+p0.5p,black+c3p')
fig.show()


# Setup CSV's in format for inversion

In [ ]:
def edit_csvs(file_in, file_out, rho):
    df = pd.read_csv(file_in, header=None, index_col=None, sep='\t', names=('X','Y','Z'))
    df['RHO']=rho
    df['CP']=0
    df.to_csv(file_out, sep=',', index=False)

In [ ]:
edit_csvs('SURFACE_1k.csv', 'ICE_1k.csv', 0.92)

In [ ]:
edit_csvs('ICEBASE_1k.csv', 'WATER_1k.csv', 1.03)

In [ ]:
edit_csvs('BED_5k.csv', 'SEDIMENT_5k.csv', 2.3)

In [ ]:
edit_csvs('BASEMENT_5k.csv', 'CRUST_5k.csv', 2.8)

In [ ]:
edit_csvs('MOHO_5k.csv', 'MANTLE_5k.csv', 3.3)

In [ ]:
df = pd.read_csv('GRAV_50k_fromgrid.csv', header=None, index_col=None, sep='\t', names=('X','Y','FA'))
df['Z']=800
df['FACP']=df.FA
df.dropna(inplace=True)
df.to_csv('GRAV_50k.csv', sep=',', index=False)
df